# Import all the Libraries and Packages

In [1]:
import os
import ee
import eemont
import geemap
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
ee.Initialize()

In [2]:
# Initialize google earth engine map
m = geemap.Map()

# Import all the Image Collections and Features then apply filter

In [3]:
# Extract landsat 5, landsat 7 and landsat 8 image collection and then filter out the unwanted pixels.

band_l8 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
band_l5_l7 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l8_bands = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(band_l8)
l7_bands = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").select(band_l5_l7)
l5_bands = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").select(band_l5_l7)
# Import Algonquin Park Boundary Shapefile for clipping and fitlering the image collection
Algonquin_Boundary = ee.FeatureCollection("projects/ee-wang25/assets/Algonquin_Boundary").geometry()

#if we also intend to filter the image caputre day of year, use "ee.Filter.dayOfYear(91,250)"
filters = [
    ee.Filter.intersects('.geo', Algonquin_Boundary),
]

l8_bands_filtered = l8_bands.filter(filters)
l7_bands_filtered = l7_bands.filter(filters)
l5_bands_filtered = l5_bands.filter(filters)

In [4]:
# Apply the cloud, shadow and snow mask

def getBit(n):
    # Returns a GEE server-side object representing `int(2^n)`
    return ee.Number(2).pow(n).int()


def addMaskBand(image):   
    qa = image.select("QA_PIXEL")
    
    dilatedCloudBit = getBit(1)
    cirrusBit = getBit(2)
    cloudBit = getBit(3)
    cloudShadowBit = getBit(4)
    snowBit = getBit(5)
    
    # Define the mask by extracting these bits and reclassifying the pixel based on the bit's value
    mask = ee.Image(0)\
        .where(qa.bitwiseAnd(dilatedCloudBit).neq(0), 1)\
        .where(qa.bitwiseAnd(cloudBit).neq(0), 2)\
        .where(qa.bitwiseAnd(cirrusBit).neq(0), 3)\
        .where(qa.bitwiseAnd(cloudShadowBit).neq(0), 4)\
        .where(qa.bitwiseAnd(snowBit).neq(0), 5)\
        .updateMask(image.select('QA_PIXEL').mask())\
        .rename("cloud_shadow_snow_mask")
    
    # return original image with this mask added as an extra band
    return image.addBands(mask)

l8_bands_filtered_masked = ee.ImageCollection(l8_bands_filtered).map(addMaskBand)
l7_bands_filtered_masked = ee.ImageCollection(l7_bands_filtered).map(addMaskBand)
l5_bands_filtered_masked = ee.ImageCollection(l5_bands_filtered).map(addMaskBand)

def maskImage(image):
    cloud_shadow_snow = image.select("cloud_shadow_snow_mask")
    return image.updateMask(cloud_shadow_snow.eq(0))

l8_final = l8_bands_filtered_masked.map(maskImage)
l7_final = l7_bands_filtered_masked.map(maskImage)
l5_final = l5_bands_filtered_masked.map(maskImage)

# Calculate the Tasseled Cap Index for Landsat 5 & 7 - Crist and Kauth (1986)

In [5]:
# Define functions to calculate brightness, greeness and wetness individually and then add each band to the image collection

def calcTasseledCapIndex_Brightness(image):
    brightness = image.expression(
        "(0.2909 * TM1) + (0.2493 * TM2) + (0.4806 * TM3) + (0.5568 * TM4) + (0.4438 * TM5) + (0.1706 * TM7)", {
            "TM1" : image.select("SR_B1"),
            "TM2" : image.select("SR_B2"),
            "TM3" : image.select("SR_B3"),
            "TM4" : image.select("SR_B4"),
            "TM5" : image.select("SR_B5"),
            "TM7" : image.select("SR_B7")
    }).rename("brightness")
    return image.addBands(brightness)

def calcTasseledCapIndex_Greeness(image):
    greeness = image.expression(
        "- (0.2728 * TM1) - (0.2174 * TM2) - (0.5508 * TM3) + (0.7221 * TM4) + (0.0733 * TM5) - (0.1648 * TM7)", {
            "TM1" : image.select("SR_B1"),
            "TM2" : image.select("SR_B2"),
            "TM3" : image.select("SR_B3"),
            "TM4" : image.select("SR_B4"),
            "TM5" : image.select("SR_B5"),
            "TM7" : image.select("SR_B7")
    }).rename("greeness")
    return image.addBands(greeness)

def calcTasseledCapIndex_Wetness(image):
    wetness = image.expression(
        "(0.1446 * TM1) + (0.1761 * TM2) + (0.3322 * TM3) + (0.3396 * TM4) - (0.6210 * TM5) - (0.4186 * TM7)", {
            "TM1" : image.select("SR_B1"),
            "TM2" : image.select("SR_B2"),
            "TM3" : image.select("SR_B3"),
            "TM4" : image.select("SR_B4"),
            "TM5" : image.select("SR_B5"),
            "TM7" : image.select("SR_B7")
    }).rename("wetness")
    return image.addBands(wetness)

In [6]:
# Test run on landsat 5 image collection
l5_final_B = ee.ImageCollection(l5_final).map(calcTasseledCapIndex_Brightness)
l5_final_BG = ee.ImageCollection(l5_final_B).map(calcTasseledCapIndex_Greeness)
l5_final_BGW = ee.ImageCollection(l5_final_BG).map(calcTasseledCapIndex_Wetness)

#Test if the first image does include all three Brightness, Greeness and Wetness Bands
ee.Image(l5_final_BGW.first()).bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B7',
 'QA_PIXEL',
 'cloud_shadow_snow_mask',
 'brightness',
 'greeness',
 'wetness']

In [7]:
# Test run on landsat 7 image collection
l7_final_B = ee.ImageCollection(l7_final).map(calcTasseledCapIndex_Brightness)
l7_final_BG = ee.ImageCollection(l7_final_B).map(calcTasseledCapIndex_Greeness)
l7_final_BGW = ee.ImageCollection(l7_final_BG).map(calcTasseledCapIndex_Wetness)

#Test if the first image does include all three Brightness, Greeness and Wetness Bands
ee.Image(l7_final_BGW.first()).bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B7',
 'QA_PIXEL',
 'cloud_shadow_snow_mask',
 'brightness',
 'greeness',
 'wetness']

In [8]:
# Visualization

TasseledCapParams = {
  "bands": ['brightness', 'greeness', 'wetness'],
    "min": [0, 0, 0],
    "max": [30000, 20000, 20000]
}

m.addLayer(l5_final_BGW, TasseledCapParams, "l5_final_BGW")
m.addLayer(l7_final_BGW, TasseledCapParams, "l7_final_BGW")
m.setCenter(-78.3, 45.77, 9)

# Calculate the Tasseled Cap Index for Landsat 8 - Baig et al. (2014)

In [9]:
# Define functions to calculate brightness, greeness and wetness individually and then add each band to the image collection

def calcTasseledCapIndex_Brightness(image):
    brightness = image.expression(
        "(0.3029 * TM1) + (0.2786 * TM2) + (0.4733 * TM3) + (0.5599 * TM4) + (0.508 * TM5) + (0.1872 * TM7)", {
            "TM1" : image.select("SR_B2"),
            "TM2" : image.select("SR_B3"),
            "TM3" : image.select("SR_B4"),
            "TM4" : image.select("SR_B5"),
            "TM5" : image.select("SR_B6"),
            "TM7" : image.select("SR_B7")
    }).rename("brightness")
    return image.addBands(brightness)

def calcTasseledCapIndex_Greeness(image):
    greeness = image.expression(
        "- (0.2941 * TM1) - (0.243 * TM2) - (0.5424 * TM3) + (0.7276 * TM4) + (0.0713 * TM5) - (0.1608 * TM7)", {
            "TM1" : image.select("SR_B2"),
            "TM2" : image.select("SR_B3"),
            "TM3" : image.select("SR_B4"),
            "TM4" : image.select("SR_B5"),
            "TM5" : image.select("SR_B6"),
            "TM7" : image.select("SR_B7")
    }).rename("greeness")
    return image.addBands(greeness)

def calcTasseledCapIndex_Wetness(image):
    wetness = image.expression(
        "(0.1511 * TM1) + (0.1973 * TM2) + (0.3283 * TM3) + (0.3407 * TM4) - (0.7117 * TM5) - (0.4559 * TM7)", {
            "TM1" : image.select("SR_B2"),
            "TM2" : image.select("SR_B3"),
            "TM3" : image.select("SR_B4"),
            "TM4" : image.select("SR_B5"),
            "TM5" : image.select("SR_B6"),
            "TM7" : image.select("SR_B7")
    }).rename("wetness")
    return image.addBands(wetness)

In [10]:
# Test run on landsat 7 image collection
l8_final_B = ee.ImageCollection(l8_final).map(calcTasseledCapIndex_Brightness)
l8_final_BG = ee.ImageCollection(l8_final_B).map(calcTasseledCapIndex_Greeness)
l8_final_BGW = ee.ImageCollection(l8_final_BG).map(calcTasseledCapIndex_Wetness)

#Test if the first image does include all three Brightness, Greeness and Wetness Bands
ee.Image(l8_final_BGW.first()).bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'QA_PIXEL',
 'cloud_shadow_snow_mask',
 'brightness',
 'greeness',
 'wetness']

In [11]:
# Visualization

TasseledCapParams = {
  "bands": ['brightness', 'greeness', 'wetness'],
    "min": [0, 0, 0],
    "max": [30000, 20000, 20000]
}

m.addLayer(l8_final_BGW, TasseledCapParams, "l8_final_BGW")
m.setCenter(-78.3, 45.77, 9)

# Select the Wetness Bands and merge them together

In [12]:
# Select all the wetness bands

l5_Wetness = ee.ImageCollection(l5_final_BGW).select("wetness")
l7_Wetness = ee.ImageCollection(l7_final_BGW).select("wetness")
l8_Wetness = ee.ImageCollection(l8_final_BGW).select("wetness")

# Merge the Wetness Index for Landsat 5, 7, 8
l5_7_Wetness = l5_Wetness.merge(l7_Wetness)
l5_7_8_Wetness = l5_7_Wetness.merge(l8_Wetness)

In [13]:
l5_7_8_Wetness.first().bandNames().getInfo()
print(l5_7_8_Wetness.size().getInfo())

3782


In [14]:
# Display the wetness index

WetnessParams = {
  'min': -2500,
  'max': 2500,
  'palette': ['#762a83','#9970ab','#c2a5cf','#e7d4e8','#f7f7f7','#d9f0d3','#a6dba0','#5aae61','#1b7837']
}

m = geemap.Map()
m.addLayer(l5_7_8_Wetness, WetnessParams, "Landsat Tasseled Cap Wetness Index")
m

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Using Geopandas to Loop through Shapefile and Export Tasseled Cap Index Value

In [15]:
import geopandas as gpd

# Use ArcGIS Pro to modify this file or to select more pixels
# ArcGIS Pro: Upperstream Location
# https://www.lioapplications.lrc.gov.on.ca/OWIT/index.html?viewer=OWIT.OWIT&locale=en-CA
# Note: For Test_Upperstream_Location, the attribute "id" is now "Id". Make sure to change that in the test run.
upstream = gpd.read_file("D:/Yannans Stuff/Data/Visible Dam/Visible Dam.shp")
upstream

,Id,geometry
0,0,POINT (-78.68041 45.69172)
1,1,POINT (-78.67734 45.69231)
2,2,POINT (-78.67780 45.68798)


In [16]:
for i in upstream.index:
    print(upstream.loc[i,'Id'])

0
1
2


In [17]:
# outdir = "D:/Yannans Stuff/Data/NDWI_Export" # specify an output directory
# os.makedirs(outdir, exist_ok = True) ## create it if it doesn't already exist

In [19]:
for j in range(0, len(upstream)):
    i = upstream.index[j]
    # Here we will convert each individual point into a ee.geometry
    gpd_geom = upstream.loc[i,'geometry']
    x = gpd_geom.xy[0][0]
    y = gpd_geom.xy[1][0]
    ee_geom = ee.Geometry.Point([x,y])
    
    # Followed by the eemont code
    ts = l5_7_8_Wetness.getTimeSeriesByRegion(geometry = ee_geom,
                               reducer = ee.Reducer.mean(),
                               scale = 30)
    tsPandas = geemap.ee_to_pandas(ts)
    tsPandas[tsPandas == -9999] = np.nan
    tsPandas = tsPandas.dropna()
    
    #Create an output label using the output directory and the 'id' column
    ID = upstream.loc[i,'Id']
    tsPandas.to_csv(r"D:\Yannans Stuff\Data\Visible Dam\Landsat5_7_8_TasseledCapIndex_TS_{0}.csv".format(ID))